In [1]:
from yahoo_oauth import OAuth2
import pandas as pd
import numpy as np
import yahoo_fantasy_api as yfa

#https://yahoo-fantasy-api.readthedocs.io/en/latest/yahoo_fantasy_api.html#the-league-class

In [2]:
conn = OAuth2(None, None, from_file='private.json')
if not conn.token_is_valid():
    conn.refresh_access_token()

[2020-08-16 20:37:14,852 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2020-08-16 20:37:14,855 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 12524.21569609642
[2020-08-16 20:37:14,855 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2020-08-16 20:37:14,856 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2020-08-16 20:37:15,166 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 0.30971598625183105
[2020-08-16 20:37:15,168 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [6]:
league = yfa.League(conn,'nfl.l.254924')
free_agents = pd.DataFrame(league.free_agents('RB'))
##get the free agents
list_free_agents = list(free_agents['player_id'])

#players = pd.DataFrame(league.player_stats(list_free_agents, 'season', season=2019))
#pd.DataFrame(league.player_details('Lamar Jackson'))



In [13]:
# Grab season stats and calculate points
# Join points to prior season

thisSeason = 2018
priorSeason = thisSeason - 1

points = {
    'Pass Yds': 0.04,
    'Pass TD': 4,
    'Int': -1,
    'Rush Att': 0.1,
    'Rush TD': 6,
    'Rec': 1,
    'Rec Yds': 0.1,
    'Rec TD': 6,
    'Ret TD': 6,
    '2-PT': 2,
    'Fum Lost': -2,
    'Fum Ret TD': 2
}

thisSeasonStats = pd.DataFrame(league.player_stats(list_free_agents, 'season', season = thisSeason))

# Calculate offensive points
thisSeasonStats['Points'] = thisSeasonStats['Pass Yds'] * points['Pass Yds'] + thisSeasonStats['Pass TD'] * points['Pass TD'] + thisSeasonStats['Int'] * points['Int'] + thisSeasonStats['Rush Att'] * points['Rush Att'] + thisSeasonStats['Rush TD'] * points['Rush TD'] + thisSeasonStats['Rec'] * points['Rec'] + thisSeasonStats['Rec Yds'] * points['Rec Yds'] + thisSeasonStats['Rec TD'] * points['Rec TD'] + thisSeasonStats['Ret TD'] * points['Ret TD'] + thisSeasonStats['2-PT'] * points['2-PT'] + thisSeasonStats['Fum Lost'] * points['Fum Lost'] + thisSeasonStats['Fum Ret TD'] * points['Fum Ret TD'] 

thisSeasonPoints = thisSeasonStats[['player_id','Points']].copy()
thisSeasonPoints

lastSeasonStats = pd.DataFrame(league.player_stats(list_free_agents, 'season', season = priorSeason))

In [14]:
# join both tables together
df = thisSeasonPoints.merge(lastSeasonStats, on = 'player_id')

# drop name
df.drop(columns = ['name'])

,player_id,Points,position_type,Pass Yds,Pass TD,Int,Rush Att,Rush Yds,Rush TD,Rec,Rec Yds,Rec TD,Ret TD,2-PT,Fum Lost,Fum Ret TD,Targets
0,7241,46.0,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8261,109.9,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9317,91.9,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24760,42.7,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24815,91.8,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,33281,0.0,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,33297,0.0,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,33309,0.0,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,33345,0.0,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
